In [1]:
# (_.~" IMPORTS "~._) 

from urllib.request import urlopen
from bs4 import BeautifulSoup as bs
from datetime import datetime
import numpy as np
import pandas as pd
import re

# ---

# (_.~" METHODES "~._)  

# NETTOYAGE SIMPLE DES BALISES

def simpleClean(toClean) :

    # Instance de finalClean
    finalClean = []
    
    # Nettoyage de chaque entitée de la liste
    for thingToClean in toClean :
        to_clean = re.compile('<.*?>')
        toCleanStr = str(thingToClean)
        toCleanStr = re.sub(to_clean, '',toCleanStr)
        finalClean.append(toCleanStr)
    
    # Remplacement de l'ancienne liste par la liste clean
    return finalClean

# ---

# (_.~" DECLARATIONS VARIABLES "~._) 

# Instance du DataFrame principal

df = pd.DataFrame(columns=['Date','Hardware','Units','Softwares'])

# Instance des listes pour le scrapping

yearList = ["2017","2018","2019","2020","2021","2022","2023"]
monthList = ["02","05","08","11"]

# Déclaration des variables nécessaires aux URLs où la place 02 et 04
# sont réservés pour les dates et les hardwares

url01 = "https://web.archive.org/web/"
url03 = "15120000/https://www.nintendo.co.jp/ir/en/finance/hard_soft/index.html"

# Variables nécessaire à la création d'URL

urlList = []
url = ""

# ---

# (_.~" CODE "~._)  

print("\n")
print("_ - _ SCRAPPING _ - _")
print("\n")
print("Boucle en cour :")

for year in yearList:
        
    # Pour savoir où on se situe en scrapping
    print(year)
        
    for month in monthList:
        url = url01 + year + month + url03
            
        # (_.~" SCRAPPIIIIING TIME "~._)
   
        # Récupération et stockage du code HTML
        page = urlopen(url)
        soup = bs(page, "html.parser")
            
        # ---
    
        # oOoOoOo TEXT MINING DES DONNEES oOoOoOo
    
        # Récupération de la date de mis à Jour des données selon Nintendo
        date = soup.find('div', attrs = {'class' : 'sinf_date'}).string
    
        # Recupération de l'année et du mois de la mise à jour
        dateUpdateStr = date[8:-2]
        dateUpdate = datetime.strptime(dateUpdateStr, '%B %d, %Y')

        # Instance des listes contenant les titres et les ventes
        hardwares = []
        allSales = []
        hardSales = []
        softSales = []

        # Récupération des données
        hardwares = soup.findAll('p', attrs = {'class' : "hard_name"})
        allSales = soup.findAll('span', attrs = {'class' : "num"})
        # Nettoyage des données
        hardwares = simpleClean(hardwares)
        allSales = simpleClean(allSales)
        
        switch = True
        
        for number in allSales:
            if switch == True:
                hardSales.append(float(number))
                switch = False
            else:
                softSales.append(float(number))
                switch = True
        
        # ---
    
        # oOoOoOo CREATION DU DATAFRAME oOoOoOo
            
        # Création des données à insérer dans les colonnes Date et Console
        tempDateList = [dateUpdate for i in range(len(hardwares))]

    
        # Création du DataFrame contenant les lignes à ajouter
        dfSoup = pd.DataFrame(list(zip(tempDateList, hardwares, hardSales, softSales)), 
                              columns=['Date','Hardware','Units','Softwares'])

        # Fusion des deux DataFrame
        df = pd.concat([df, dfSoup])
            

# ---

print("Scrapping terminé !")
print("\n")

# (_.~" NETTOYAGE PRIMAIRE DU DATAFRAME "~._) 

print("_ - _ DATAFRAME _ - _")
print("\n")
print("Check avant export :")

# Reset de l'index
df.reset_index(drop=True, inplace=True)

# Doublons check avant suppression
doublons = df.duplicated().sum()
if doublons > 0:
    print(doublons," doublons présents dans le DataFrame.")
    df.drop_duplicates(inplace=True)
    doublons = df.duplicated().sum()
    if doublons > 0:
        print(doublons," doublons restent après suppression dans le DataFrame.")
    else:
        print("Tous les doublons ont été supprimés !")
else:
    print("Il n'y a aucun doublon dans le DataFrame.")
    
# NaN check
nanDfTotal = df.isna().sum().sum()
nanDf = df.isna().sum()
if nanDfTotal > 0:
    print("Données manquantes dans le DataFrame")
    print(nanDf)
else:
    print("Il n'y a aucune données manquante dans le DataFrame.")
    
# ---

# (_.~" EXPORT "~._) 

# Export du Dataframe dans un fichier CSV

print("\n")
print("Export en cours :")

csvName = 'nintendo_hardware_sales.csv'

df.to_csv(csvName, index=False)
print("Export réussi sous :", csvName)



_ - _ SCRAPPING _ - _


Scrap en cours :
 100 % effectués ( 2023 11 )
 C'est terminé !

324 données collectées. 

_ - _ DATAFRAME _ - _


Check avant export :
9  doublons présents dans le DataFrame.
Tous les doublons ont été supprimés !
Il n'y a aucune données manquante dans le DataFrame.


Export en cours :
Export réussi sous : nintendo_hardware_sales.csv
